In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import quandl
import cvxpy as cvx

In [3]:
sys.path.insert(0, os.path.abspath('..'))
import helium

In [4]:
%matplotlib inline

In [5]:
quandl.ApiConfig.api_key = '8wnB7EcvJm1xEvJjzWBR'

In [6]:
tickers = ['AMZN', 'GOOGL', 'MCD', 'NKE']
start_date='2016-01-01'
end_date='2017-03-13'
prices = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Close'])
                for ticker in tickers]))

returns=prices.pct_change()
returns[["_CASH"]]=quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date)/(250*100)
returns = returns.fillna(method='ffill').iloc[1:]

returns.tail()

,AMZN,GOOGL,MCD,NKE,_CASH
Date,,,,,
2017-03-07,-0.000697,0.004579,0.000312,-0.003875,0.000030
2017-03-08,0.005295,0.002925,0.000156,-0.000707,0.000029
2017-03-09,0.002939,0.004920,0.000390,-0.002654,0.000029
2017-03-10,-0.000633,0.004156,-0.001249,0.001242,0.000029
2017-03-13,0.002499,0.003686,-0.002891,0.004253,0.000031


In [7]:
rets = returns.rolling(window=250, min_periods=250).mean().shift(1).dropna()
sigmas = returns.rolling(window=250, min_periods=250).cov().unstack().shift(1).stack()

rets.tail()

,AMZN,GOOGL,MCD,NKE,_CASH
Date,,,,,
2017-03-07,0.001749,0.000743,0.000468,-8.301724e-05,0.000014
2017-03-08,0.001752,0.000695,0.000428,4.617185e-07,0.000014
2017-03-09,0.001777,0.000669,0.000417,-4.008408e-05,0.000014
2017-03-10,0.001712,0.000619,0.000366,-1.322234e-04,0.000015
2017-03-13,0.001683,0.000607,0.000317,-1.758572e-04,0.000015


In [8]:
r_est = {}
d_est = {}
for date, row in rets.iterrows():
    r_est[date] = rets.loc[[date]].copy()
    d = rets.loc[[date]].copy()
    d[:] = 0.
    d_est[date] = d

In [9]:
cash_ticker = "_CASH"

In [10]:
price_sigma = rets.copy()
price_sigma[:] = 0.

In [11]:
volumes = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Volume'])
                for ticker in tickers]))
volumes[cash_ticker] = np.finfo(np.float).max

In [12]:
tcost_model=helium.TransactionCost(gamma = 1., 
                                   half_spread=10E-4,
                                   nonlin_coef = 0.,  #1.,
                                   sigmas = price_sigma, 
                                   nonlin_power = 1.5,
                                   volumes = volumes,
                                   asym_coef = 0.)

In [13]:
borrow_costs = rets.copy()
borrow_costs[:] = 1E-4
borrow_costs[cash_ticker] = 0.

dividends = rets.copy()
dividends[:] = 0.


hcost_model=helium.HoldingCost(gamma = 1., borrow_costs=borrow_costs, dividends=dividends)

In [14]:
risk_model = helium.BasicRiskCost(5.0, sigmas)

In [15]:
leverage_limit = helium.LeverageLimitConstraint(3.)

In [16]:
holding_init = pd.Series(index=rets.columns, data=0.)
holding_init[cash_ticker] = 10000.
holding_init

AMZN         0.0
GOOGL        0.0
MCD          0.0
NKE          0.0
_CASH    10000.0
dtype: float64

In [17]:
r = helium.DefaultReturns(returns=r_est, deltas = d_est, gamma_decay=None)

In [18]:
spo_policy = helium.SinglePeriodOpt(returns=r,
                                   costs=[risk_model, hcost_model, tcost_model],
                                   constraints=[leverage_limit]
                                   )

In [19]:
shares_to_trade=spo_policy.get_trades(h=holding_init, t=pd.to_datetime(end_date))
shares_to_trade

AMZN     3452.933557
GOOGL       0.000037
MCD         0.000034
NKE         0.000019
_CASH   -3452.933647
dtype: float64

In [20]:
np.round(shares_to_trade/prices.loc[end_date])

AMZN     4.0
GOOGL    0.0
MCD      0.0
NKE      0.0
_CASH    NaN
dtype: float64